In [1]:
from langchain.llms import GooglePalm

api_key = 'AIz##############'

llm = GooglePalm(google_api_key=api_key, temperature=0.7)

c:\ProgramData\anaconda3\envs\llm_ete_langchain_codebasics\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
poem = llm('write a 4 line poem of my love for Coffee')
print(poem)

Coffee, the nectar of the gods,
A warm embrace on a cold morning,
The smell of heaven,
The taste of love.


In [3]:
essay = llm('write email requesting refund for electric item')
print(essay)

Dear [Company Name],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [explain why you are not satisfied].

I have attached a copy of my receipt and a picture of the product. I have also included a link to the product page on your website.

I would like to request a full refund for the product. I would appreciate it if you could process my refund as soon as possible.

Thank you for your time and consideration.

Sincerely,
[Your Name]


In [6]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='codebasics_faqs.csv', source_column='prompt',encoding='latin-1')
data = loader.load()

In [12]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-base",
    query_instruction = 'Represent the query for retrieval: '
)

e = embeddings.embed_query('What is your refund policy?')

load INSTRUCTOR_Transformer
max_seq_length  512


In [13]:
e

[0.0030041553545743227,
 -0.026218092069029808,
 0.016302157193422318,
 0.05447344854474068,
 0.01621294766664505,
 -0.05103736370801926,
 -0.01582120917737484,
 0.0008341040229424834,
 -0.008621984161436558,
 -0.037801265716552734,
 -0.003327545477077365,
 -0.056575577706098557,
 0.06719810515642166,
 0.007795107085257769,
 0.07487718760967255,
 0.009460627101361752,
 -0.0007944112294353545,
 -0.006618031300604343,
 0.03602083399891853,
 0.03415404632687569,
 -0.00398759450763464,
 -0.0067665837705135345,
 -0.06024021655321121,
 0.003069438971579075,
 -0.04745917767286301,
 0.011237221769988537,
 0.019626157358288765,
 -0.023760486394166946,
 0.0025532804429531097,
 0.01835833489894867,
 0.06470326334238052,
 -0.01158981490880251,
 -0.028205079957842827,
 0.058883048593997955,
 -0.014101789332926273,
 -0.019189385697245598,
 -0.02320432849228382,
 -0.026240816339850426,
 -0.009938593953847885,
 0.019473830237984657,
 -0.009283916093409061,
 -0.0591055229306221,
 -0.040475912392139435,

In [14]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(documents=data, embedding=embeddings)

In [16]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents('how about job placement support?')
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this bootcamp guarantee me a job?', 'row': 15}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighte

In [19]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [21]:
chain('do you have javascript course?')


{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}),
  Document(page_content='prompt: What is different in this course from thousands of other Power BI courses available online?\nresponse: Most of the courses available on the internet teach you how to build x & y witho

In [22]:
chain("Do you have plans to launch blockchain course in future?")


{'query': 'Do you have plans to launch blockchain course in future?',
 'result': "We don't have any information about launching blockchain course in future.",
 'source_documents': [Document(page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}),
  Document(page_content='prompt: What business concepts and domains are covered in this course?\nresponse: We have covered the core functions such as Sales, Marketing, Finance, and Supply Chain with their fundamentals related to this course. The domain you will learn in this course is consumer goods which is projected to have more openings and high data analytics requirements at least until 2030.', metadata={'source': 'What business concepts and domains are covered in this course?', 'row': 32}),
  Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nrespons

In [23]:
chain("should I learn power bi or tableau?")


{'query': 'should I learn power bi or tableau?',
 'result': 'This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartner’s magic quadrant in BI for the last few years as the industry leader.',
 'source_documents': [Document(page_content='prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation

In [24]:
chain("I've a MAC computer. Can I use powerbi on it?")


{'query': "I've a MAC computer. Can I use powerbi on it?",
 'result': 'response: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luck with your studies!',
 'source_documents': [Document(page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luc

In [25]:
chain("I don't see power pivot. how can I enable it?")


{'query': "I don't see power pivot. how can I enable it?",
 'result': 'Follow the process in the link : \n\nhttps://drive.google.com/file/d/1-mO-v52h-YTY1s-v30liBJPu6Yj4OUxb/view?usp=share_link',
 'source_documents': [Document(page_content='prompt: How to install power pivot if its not available in system?\nresponse: Follow this thread for instructions - https://support.microsoft.com/en-us/office/start-the-power-pivot-add-in-for-excel-a891a66d-36e3-43fc-81e8-fc4798f39ea8\nIf it doesn\'t show in the ribbon then go to "insert" tab. You will be able to see pivot table option there.', metadata={'source': 'How to install power pivot if its not available in system?', 'row': 38}),
  Document(page_content='prompt: How do I enable Power Pivot before using it for the first time ?\nresponse: Follow the process in the link : \n\nhttps://drive.google.com/file/d/1-mO-v52h-YTY1s-v30liBJPu6Yj4OUxb/view?usp=share_link', metadata={'source': 'How do I enable Power Pivot before using it for the first time

In [26]:
chain("What is the price of your machine learning course?")

{'query': 'What is the price of your machine learning course?',
 'result': 'response: 13999 INR (199 USD)',
 'source_documents': [Document(page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}),
  Document(page_content='prompt: I\x92m not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don\x92t worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the course instructor) runs a popular data science YouTube channel called Codebasics. On that, you can watch his videos and read comments to get an idea of his teaching style', metadata={'source': 'I\x92m not sure if this course is good enough for me to invest some money. What can I do?', 'row': 20}),
  Document(page_content='prompt: What is different in this course compared to hundreds of courses on t